# 1. Configurar ambiente

In [1]:
import pandas as pd
import numpy as np
import os
import re
import spacy
#!python -m spacy download es_core_news_md
nlp = spacy.load('es_core_news_md')
import jellyfish
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
#from bs4 import BeautifulSoup
#import requests
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


e:\Coding\chatbot\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#2. Tratamiento de datos

In [2]:
#Función para encontrar la raiz de las palabras
def raiz(palabra):
  radio=0
  palabra_encontrada=palabra
  for word in lista_verbos:
    confianza = jellyfish.jaro_winkler(palabra, word)
    if (confianza>=0.93 and confianza>=radio):
      radio=confianza
      palabra_encontrada=word
  return palabra_encontrada

#Función para tratar los textos
def tratamiento_texto(texto):
  trans = str.maketrans('áéíóú','aeiou')
  texto = texto.lower()
  texto = texto.translate(trans)
  texto = " ".join(texto.split())
  return texto

#Función para reemplazar el final de una palabra por 'r'
def reemplazar_terminacion(palabra):
  patron = r"(es|me|as|te|ste)$"
  nueva_palabra = re.sub(patron, "r", palabra)
  return nueva_palabra.split()[0]

#Función para adicionar o eliminar tokens
def revisar_tokens(texto, tokens):
  if len(tokens)==0:
    if [x for x in ['Geo Perú', 'geoperu', 'GeoPeru', 'geo peru'] if x in tratamiento_texto(texto)]: tokens.append('GeoPerú')
    elif [x for x in ['geo referencia', 'georreferencial'] if x in tratamiento_texto(texto)]: tokens.append('georreferencias')
  else:
    elementos_a_eliminar = ["cual", "que", "quien", "cuanto", "cuando", "como"]
    if 'hablame' in texto and 'hablar' in tokens: tokens.remove('hablar')
    elif 'cuentame' in texto and 'contar' in tokens: tokens.remove('contar') 
    elif 'hago' in texto and 'hacer' in tokens: tokens.remove('hacer') 
    elif 'entiendes' in texto and 'entender' in tokens: tokens.remove('entender') 
    elif 'sabes' in texto and 'saber' in tokens: tokens.remove('saber')
    tokens = [x.replace('datar','data').replace('datos','dato') for x in tokens if x not in elementos_a_eliminar]
  return tokens

#Función para devolver los tokens normalizados del texto
def normalizar(texto):
  tokens=[]
  tokens=revisar_tokens(texto, tokens)
  if 'geoperu' in tokens:
    texto = ' '.join(texto.split()[:15])
  else:
    texto = ' '.join(texto.split()[:25])

  doc = nlp(texto)
  for t in doc:
    lemma=diccionario_irregulares.get(t.text, t.lemma_.split()[0])
    lemma=re.sub(r'[^\w\s+\-*/]', '', lemma)
    if t.pos_ in ('VERB','PROPN','PRON','NOUN','AUX','SCONJ','ADJ','ADV','NUM') or lemma in lista_verbos:
      if t.pos_=='VERB':
        lemma = reemplazar_terminacion(lemma)
        tokens.append(raiz(tratamiento_texto(lemma)))
      else:
        tokens.append(tratamiento_texto(lemma))

  tokens = list(dict.fromkeys(tokens))
  tokens = list(filter(None, tokens))
  tokens = revisar_tokens(texto, tokens)
  tokens = ' '.join(tokens)
  return tokens

# 3. Cargar bases de conocimiento

In [3]:

#Importando verbos en español

trans = str.maketrans('áéíóú','aeiou')
ruta_archivo = os.path.join(os.getcwd(), 'verbos.txt')
lista_verbos = []
with open(ruta_archivo, 'r', encoding='utf-8') as archivo:
  contenido = archivo.read().strip()
lineas = contenido.split('\n')
for linea in lineas:
  palabra = linea.strip()
  lista_verbos.append(palabra)
nuevos_verbos = ['costar', 'referir', 'datar']
lista_verbos.extend(nuevos_verbos)
lista_verbos=list(set(lista_verbos))

# Definir una lista de verbos irregulares y sus conjugaciones en pasado, presente, futuro, imperfecto, pretérito y condicional
verbos_irregulares = [
    ('ser', 'soy', 'eres', 'seras', 'eras', 'es', 'serias'),
    ('estar', 'estuviste', 'estas', 'estaras', 'estabas', 'estuviste', 'estarias'),
    ('ir', 'fuiste', 'vas', 'iras', 'ibas', 'fuiste', 'irias'),
    ('ir', 'fuiste', 'vaya', 'iras', 'ibas', 'fuiste', 'irias'),
    ('tener', 'tuviste', 'tienes', 'tendras', 'tenias', 'tuviste', 'tendrias'),
    ('hacer', 'hiciste', 'haces', 'haras', 'hacias', 'hiciste', 'harias'),
    ('decir', 'dijiste', 'dices', 'diras', 'decias', 'dijiste', 'dirias'),
    ('decir', 'dimar', 'dime', 'digame', 'dimir', 'dimo', 'dimiria'),
    ('poder', 'pudiste', 'puedes', 'podras', 'podias', 'pudiste', 'podrias'),
    ('saber', 'supiste', 'sabes', 'sabras', 'sabias', 'supiste', 'sabrias'),
    ('poner', 'pusiste', 'pones', 'pondras', 'ponias', 'pusiste', 'pondrias'),
    ('ver', 'viste', 'ves', 'veras', 'veias', 'viste', 'verias'),
    ('dar', 'diste', 'das', 'daras', 'dabas', 'diste', 'darias'),
    ('dar', 'damar', 'dame', 'daras', 'dabas', 'darme', 'darias'),
    ('venir', 'viniste', 'vienes', 'vendras', 'venias', 'viniste', 'vendrias'),
    ('haber', 'haya', 'has', 'habras', 'habias', 'hubiste', 'habrias'),
    ('caber', 'cupiste', 'cabes', 'cabras', 'cabias', 'cupiste', 'cabrias'),
    ('valer', 'valiste', 'vales', 'valdras', 'valias', 'valiste', 'valdrias'),
    ('querer', 'quisiste', 'quieres', 'querras', 'querias', 'quisiste', 'querrias'),
    ('llegar', 'llegaste', 'llegares', 'llegaras', 'llegarias', 'llegaste', 'llegarrias'),
    ('hacer', 'hiciste', 'haces', 'haras', 'hacias', 'hiciste', 'harias'),
    ('decir', 'dijiste', 'dices', 'diras', 'decias', 'dijiste', 'dirias'),
    ('poder', 'pudiste', 'puedes', 'podras', 'podias', 'pudiste', 'podria'),
    ('contar', 'contaste', 'cuentas', 'contaras', 'contabas', 'cuentame', 'contarias'),
    ('saber', 'supiste', 'sabes', 'sabras', 'sabias', 'supiste', 'sabrias'),
    ('costar', 'cuesta', 'cuestan', 'costo', 'costaria', 'costarian', 'cuestas'),
    ('durar', 'duraste', 'duro', 'duraras', 'durabas', 'duraste', 'durarias')
]

# Crear el DataFrame
diccionario_irregulares = {}
df = pd.DataFrame(verbos_irregulares, columns=['Verbo', 'Pasado', 'Presente', 'Futuro', 'Imperfecto', 'Pretérito', 'Condicional'])
for columna in df.columns:
  if columna != 'Verbo':
    for valor in df[columna]:
      diccionario_irregulares[valor] = df.loc[df[columna] == valor, 'Verbo'].values[0]

#Importando bases de dialogo fluído
txt_folder_path = os.path.join(os.getcwd(),'txt')
lista_documentos=[x for x in os.listdir(txt_folder_path) if x.endswith(".txt")]
lista_dialogos, lista_dialogos_respuesta, lista_tipo_dialogo = [],[],[]
for idx in range(len(lista_documentos)):
  f=open(txt_folder_path+'/'+lista_documentos[idx], 'r', encoding='utf-8', errors='ignore')
  flag,posicion = True,0
  for line in f.read().split('\n'):
    if flag:
      line = tratamiento_texto(line)
      line = re.sub(r'[^\w\s+\-*/]', '', line)
      lista_dialogos.append(line)
      lista_tipo_dialogo.append(lista_documentos[idx].replace('.txt', ''))
    else:
      lista_dialogos_respuesta.append(line)
      posicion+=1
    flag=not flag
#Creando Dataframe de diálogos
datos = {'dialogo':lista_dialogos,'respuesta':lista_dialogos_respuesta,'tipo':lista_tipo_dialogo,'interseccion':0,'similarity':0,'jaro_winkler':0,'probabilidad':0}
df_dialogo = pd.DataFrame(datos)
df_dialogo = df_dialogo.drop_duplicates(keep='first')
df_dialogo.reset_index(drop=True, inplace=True)

In [4]:
print(lista_verbos)
print(len(lista_verbos))

['alegrarse', 'verificar', 'envidiar', 'odiar', 'reprochar', 'wasapees', 'asociar', 'conocer', 'mitificar', 'sumergir', 'ir', 'hinchar', 'extender', 'obedecer', 'detectar', 'nacionalizar', 'nivelar', 'diferenciar', 'cortar', 'amaria', 'noctambular', 'ablandar', 'asomar', 'legalizar', 'datar', 'anhelar', 'restituir', 'remplazar', 'eliminar', 'habituar', 'avisar', 'emplear', 'socorrer', 'maldecir', 'ulcerar', 'violar', 'fabricar', 'holgar', 'medicar', 'estremecer', 'ilusionar', 'acoplar', 'regir', 'malograr', 'explicitar', 'salir', 'operar', 'agregar', 'favorecer', 'inspirar', 'completar', 'subyacer', 'equilibrar', 'suplir', 'expirar', 'influir', 'afinar', 'simbolizar', 'reflejar', 'ellas', 'adoptar', 'repercutir', 'consultar', 'expatriar', 'pedalear', 'abrir', 'traquetear', 'cobrar', 'ingerir', 'wasapeaban', 'empeñar', 'enchufar', 'habilitar', 'elegir', 'reflexionar', 'saborear', 'activar', 'intuyo', 'herrar', 'sufrir', 'amarais o amaseis', 'trinar', 'fregar', 'burlar', 'consolidar', 'a

# 4. Normalizando dialogos

In [5]:
#Normalizando las frases
label_encoder = LabelEncoder()
df_dialogo['palabras'] = df_dialogo['dialogo'].apply(normalizar)
df_dialogo['tipo_num'] = label_encoder.fit_transform(df_dialogo['tipo'])
df_dialogo = df_dialogo[df_dialogo.palabras.values!='']
df_dialogo

,dialogo,respuesta,tipo,interseccion,similarity,jaro_winkler,probabilidad,palabras,tipo_num
0,bueno te elogio a ti,¡Muchas gracias! Me alegro mucho de oír eso.,Agradecimiento,0,0,0,0,bueno tu elogio,0
1,de acuerdo,¡Gracias!,Agradecimiento,0,0,0,0,acuerdo,0
2,entiendo,¡Perfecto! ¡Me alegra que hayas entendido!,Agradecimiento,0,0,0,0,entender,0
3,estoy de acuerdo,¡Gracias!,Agradecimiento,0,0,0,0,estar acuerdo,0
4,igualmente,¡Igualmente!,Agradecimiento,0,0,0,0,igualmente,0
...,...,...,...,...,...,...,...,...,...
1129,que medidas tomas para proteger la seguridad d...,"Como chatbot, no recolecto ni almaceno informa...",Usuario,0,0,0,0,medida tomar proteger seguridad informacion pe...,16
1130,sabes cual es mi direccion de residencia,"Como chatbot, no tengo acceso ni almaceno info...",Usuario,0,0,0,0,ser direccion residencia,16
1131,almacenas mi nombre de usuario y contraseña en...,"Como chatbot, no tengo la capacidad de almacen...",Usuario,0,0,0,0,almacenar nombre usuario contraseña sistema,16
1132,que haces con mi informacion personal,"Como chatbot, no almaceno ni uso información p...",Usuario,0,0,0,0,hacer informacion personal,16


In [6]:
# Imprimir diccionario
relacion_diccionario = {}

# Iterar sobre las filas del DataFrame
for tipo, tipo_num in zip(df_dialogo['tipo'], df_dialogo['tipo_num']):
    relacion_diccionario[tipo_num] = tipo

# Imprimir el diccionario
print(relacion_diccionario)

{0: 'Agradecimiento', 1: 'Aprendizaje', 2: 'Contacto', 3: 'Continuacion', 4: 'Despedida', 5: 'Edad', 6: 'Error', 7: 'Funcion', 8: 'GeoPeru', 9: 'Identidad', 10: 'Nombre', 11: 'Origen', 12: 'Otros', 13: 'Qa', 14: 'Saludos', 15: 'Sentimiento', 16: 'Usuario'}


# 5. Entrenando con Naive Bayes

In [ ]:
# Separar los datos en características (X) y etiquetas (y)
X = df_dialogo['palabras']
y = df_dialogo['tipo_num']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorizar los datos de texto
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Entrenar el clasificador de Naive Bayes
modelo_NB = MultinomialNB()
modelo_NB.fit(X_train_vect, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = modelo_NB.predict(X_test_vect)

# Calcular el accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Precisión:", accuracy)

Precisión: 0.7


In [ ]:
# Calcular la precisión por clase
unique_classes = df_dialogo['tipo_num'].unique()
for cls in unique_classes:
    cls_indices = y_test == cls
    cls_accuracy = accuracy_score(y_test[cls_indices], y_pred[cls_indices])
    print("Accuracy para la clase", df_dialogo[df_dialogo.tipo_num == cls]['tipo'].unique()[0], ":", cls_accuracy)

Accuracy para la clase Continuacion : 0.5
Accuracy para la clase Nombre : 0.16666666666666666
Accuracy para la clase Contacto : 0.125
Accuracy para la clase Saludos : 0.9117647058823529
Accuracy para la clase Sentimiento : 0.45454545454545453
Accuracy para la clase Identidad : 0.6153846153846154
Accuracy para la clase Usuario : 0.7
Accuracy para la clase ElProfeAlejo : 0.5
Accuracy para la clase Aprendizaje : 0.2
Accuracy para la clase Agradecimiento : 0.7
Accuracy para la clase Edad : 0.5
Accuracy para la clase Despedida : 0.5
Accuracy para la clase Origen : 0.5
Accuracy para la clase Otros : 0.9565217391304348
Accuracy para la clase Error : 0.5
Accuracy para la clase Funcion : 0.5333333333333333


In [ ]:
# Procesando la nueva frase
frase = normalizar('como haces para aprender tan rapido?')
nueva_frase_vect = vectorizer.transform([frase])

# Realizar la predicción
prediccion = modelo_NB.predict(nueva_frase_vect)

print("La frase", frase, "se clasifica como: ", df_dialogo[df_dialogo.tipo_num == prediccion[0]]['tipo'].unique()[0])

La frase como hacer aprender tanto rapido se clasifica como:  Origen


# 6. Entrenando con Random Forest

In [7]:
# Separar los datos en características (X) y etiquetas (y)
X = df_dialogo['palabras']
y = df_dialogo['tipo_num']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorizar los datos de texto
vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

# Entrenar el clasificador Random Forest
Modelo_RF = RandomForestClassifier()
Modelo_RF.fit(X_train_vect, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = Modelo_RF.predict(X_test_vect)

# Calcular el accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Precisión:", accuracy)

Precisión: 0.7149122807017544


In [8]:
# Calcular la precisión por clase
unique_classes = df_dialogo['tipo_num'].unique()
for cls in unique_classes:
    cls_indices = y_test == cls
    cls_accuracy = accuracy_score(y_test[cls_indices], y_pred[cls_indices])
    print("Accuracy para la clase", df_dialogo[df_dialogo.tipo_num == cls]['tipo'].unique()[0], ":", cls_accuracy)

Accuracy para la clase Agradecimiento : 0.5714285714285714
Accuracy para la clase Aprendizaje : 0.5833333333333334
Accuracy para la clase Contacto : 0.25
Accuracy para la clase Continuacion : 0.5714285714285714
Accuracy para la clase Despedida : 0.6
Accuracy para la clase Edad : 0.6
Accuracy para la clase ElProfeAlejo : 1.0
Accuracy para la clase Error : 0.75
Accuracy para la clase Funcion : 0.8125
Accuracy para la clase Identidad : 0.8636363636363636
Accuracy para la clase Nombre : 1.0
Accuracy para la clase Origen : 0.5
Accuracy para la clase Otros : 0.6984126984126984
Accuracy para la clase Saludos : 0.9230769230769231
Accuracy para la clase Sentimiento : 0.5
Accuracy para la clase Usuario : 0.875


In [9]:
# Procesando la nueva frase
frase = normalizar('como haces para aprender tan rapido?')
nueva_frase_vect = vectorizer.transform([frase])

# Realizar la predicción
prediccion = Modelo_RF.predict(nueva_frase_vect)

print("La frase", frase, "se clasifica como: ", df_dialogo[df_dialogo.tipo_num == prediccion[0]]['tipo'].unique()[0])

La frase hacer aprender tanto rapido se clasifica como:  Funcion


In [6]:
torch.cuda.is_available()

True

# 7. Entrenando con Transformers

In [7]:
#from transformers import ElectraForPreTraining, ElectraTokenizerFast

In [8]:
# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
df_train, df_test = train_test_split(df_dialogo, test_size=0.2, random_state=42)

# Cargar el modelo preentrenado de BERT para clasificación en español
model_name = 'dccuchile/bert-base-spanish-wwm-uncased'
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=df_dialogo['tipo_num'].nunique())
tokenizer = BertTokenizer.from_pretrained(model_name)

#ELECTRA

#model_name = 'skimai/electra-small-spanish'
#model = ElectraForPreTraining.from_pretrained(model_name)
#tokenizer = ElectraTokenizerFast.from_pretrained(model_name, do_lower_case=False)

# Tokenizar y codificar las frases de entrenamiento
train_inputs = tokenizer.batch_encode_plus(
    df_train['palabras'].tolist(),
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# Tokenizar y codificar las frases de prueba
test_inputs = tokenizer.batch_encode_plus(
    df_test['palabras'].tolist(),
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# Preparar los datos de entrenamiento y prueba
train_data = torch.utils.data.TensorDataset(train_inputs['input_ids'], train_inputs['attention_mask'], torch.tensor(df_train['tipo_num'].tolist()))
test_data = torch.utils.data.TensorDataset(test_inputs['input_ids'], test_inputs['attention_mask'], torch.tensor(df_test['tipo_num'].tolist()))

# Definir el optimizador y la función de pérdida
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Entrenamiento del modelo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.train()

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=16, shuffle=True)

for epoch in range(12):  # Número de épocas de entrenamiento
    total_loss = 0

    for batch in train_dataloader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    print("Epoch:", epoch + 1, "Loss:", total_loss)

# Evaluación del modelo
model.eval()
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False)

with torch.no_grad():
    predictions = []
    true_labels = []

    for batch in test_dataloader:
        input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)

        outputs = model(input_ids, attention_mask=attention_mask)

        _, predicted_labels = torch.max(outputs.logits, dim=1)

        predictions.extend(predicted_labels.tolist())
        true_labels.extend(labels.tolist())

accuracy = accuracy_score(true_labels, predictions)
print("Precisión:", accuracy)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch: 1 Loss: 139.00864446163177
Epoch: 2 Loss: 112.87592828273773
Epoch: 3 Loss: 88.39103680849075
Epoch: 4 Loss: 68.8956795334816
Epoch: 5 Loss: 51.30662256479263
Epoch: 6 Loss: 37.98039972782135
Epoch: 7 Loss: 28.63708969950676
Epoch: 8 Loss: 22.396223410964012
Epoch: 9 Loss: 17.515371695160866
Epoch: 10 Loss: 13.534160025417805
Epoch: 11 Loss: 10.821588300168514
Epoch: 12 Loss: 8.652913697063923
Precisión: 0.8044444444444444


In [9]:
# Guardar el modelo entrenado
ruta_modelo = os.path.join(os.getcwd(),'modelo_final')
model.save_pretrained(ruta_modelo)
tokenizer.save_pretrained(ruta_modelo)

('e:\\Coding\\chatbot\\ver2\\modelo_final\\tokenizer_config.json',
 'e:\\Coding\\chatbot\\ver2\\modelo_final\\special_tokens_map.json',
 'e:\\Coding\\chatbot\\ver2\\modelo_final\\vocab.txt',
 'e:\\Coding\\chatbot\\ver2\\modelo_final\\added_tokens.json')

In [10]:
#Cargar el modelo entrenado
#ruta_modelo = os.path.join(os.getcwd(),'modelo')
Modelo_TF = BertForSequenceClassification.from_pretrained(ruta_modelo)
tokenizer_TF = BertTokenizer.from_pretrained(ruta_modelo)

In [11]:
# Calcular la precisión por clase
unique_classes = df_dialogo['tipo_num'].unique()

for class_label in unique_classes:
    # Filtrar los datos por clase
    class_data = df_dialogo[df_dialogo['tipo_num'] == class_label]

    # Preparar los datos de la clase para evaluar
    tokens = tokenizer_TF.batch_encode_plus(
        class_data['palabras'].tolist(),
        truncation=True,
        padding=True,
        return_tensors='pt'
    )

    inputs = tokens['input_ids']
    attention_mask = tokens['attention_mask']
    labels = class_data['tipo_num'].tolist()

    # Pasar los datos de la clase por el modelo
    with torch.no_grad():
        outputs = Modelo_TF(inputs, attention_mask=attention_mask)

    predicted_labels = outputs.logits.argmax(dim=1).tolist()

    # Calcular la precisión para la clase
    accuracy = accuracy_score(labels, predicted_labels)
    print(f"Precisión por clase {df_dialogo[df_dialogo.tipo_num == class_label]['tipo'].unique()[0]}: {accuracy}")

Precisión por clase Agradecimiento: 0.9545454545454546
Precisión por clase Aprendizaje: 0.9302325581395349
Precisión por clase Contacto: 0.9655172413793104
Precisión por clase Continuacion: 1.0
Precisión por clase Despedida: 1.0
Precisión por clase Edad: 0.8709677419354839
Precisión por clase Error: 1.0
Precisión por clase Funcion: 0.8985507246376812
Precisión por clase GeoPeru: 1.0
Precisión por clase Identidad: 0.975
Precisión por clase Nombre: 0.9523809523809523
Precisión por clase Origen: 0.9555555555555556
Precisión por clase Otros: 0.9290322580645162
Precisión por clase Qa: 0.6666666666666666
Precisión por clase Saludos: 0.9887005649717514
Precisión por clase Sentimiento: 1.0
Precisión por clase Usuario: 0.96875


In [13]:
# Procesar nueva frase
frase = normalizar('que es chatbot')

# Tokenizar la frase de entrada
tokens = tokenizer_TF.encode_plus(
    frase,
    add_special_tokens=True,
    max_length=128,
    padding='max_length',
    truncation=True,
    return_tensors='pt'
)

# Obtener los input_ids y attention_mask
input_ids = tokens['input_ids']
attention_mask = tokens['attention_mask']

# Realizar la predicción
with torch.no_grad():
    outputs = Modelo_TF(input_ids, attention_mask)

# Obtener las etiquetas predichas
etiquetas_predichas = torch.argmax(outputs.logits, dim=1)

# Decodificar las etiquetas predichas
etiquetas_decodificadas = etiquetas_predichas.tolist()
print("La frase", frase, "se clasifica como: ", df_dialogo[df_dialogo.tipo_num == etiquetas_decodificadas[0]]['tipo'].unique()[0])

La frase ser chatbot se clasifica como:  Identidad
